In [55]:
from dotenv import load_dotenv
import os
load_dotenv()
import requests

def usFundRequestAuthorized(resource):
    authToken = os.getenv("usFundTokenApi")
    if not authToken:
        raise ValueError("You need a token")
    else:
        print("We have a usFund token {}".format(authToken[0:4]))
    url = "https://api.usfundamentals.com{}&token=".format(resource)
    print("Requesting authorized {}".format(url))
    print(url+authToken)
    res = requests.get(url+authToken)
    return res

In [57]:
a = usFundRequestAuthorized("/v1/companies/xbrl?format=json").json()
display(a)

We have a usFund token GfgL
Requesting authorized https://api.usfundamentals.com/v1/companies/xbrl?format=json&token=
https://api.usfundamentals.com/v1/companies/xbrl?format=json&token=GfgLDIepTIST4LQpUq3_5A


[{'company_id': '1000045',
  'name_latest': 'Nicholas Financial Inc',
  'names_previous': None},
 {'company_id': '1000180',
  'name_latest': 'Sandisk Corp',
  'names_previous': None},
 {'company_id': '1000209',
  'name_latest': 'Medallion Financial Corp',
  'names_previous': None},
 {'company_id': '1000228',
  'name_latest': 'Henry Schein Inc',
  'names_previous': None},
 {'company_id': '1000229',
  'name_latest': 'Core Laboratories N V',
  'names_previous': None},
 {'company_id': '1000230',
  'name_latest': 'Optical Cable Corp',
  'names_previous': None},
 {'company_id': '1000232',
  'name_latest': 'Kentucky Bancshares Inc /KY/',
  'names_previous': None},
 {'company_id': '1000278',
  'name_latest': 'Pacifichealth Laboratories Inc',
  'names_previous': None},
 {'company_id': '1000298',
  'name_latest': 'Impac Mortgage Holdings Inc',
  'names_previous': None},
 {'company_id': '1000459',
  'name_latest': 'Green Energy Group Inc.',
  'names_previous': None},
 {'company_id': '1000623',
  

In [35]:
a = requests.get("https://api.usfundamentals.com/v1/companies/xbrl?format=json&token=GfgLDIepTIST4LQpUq3_5A").json()

In [36]:
display(a)

[{'company_id': '1000045',
  'name_latest': 'Nicholas Financial Inc',
  'names_previous': None},
 {'company_id': '1000180',
  'name_latest': 'Sandisk Corp',
  'names_previous': None},
 {'company_id': '1000209',
  'name_latest': 'Medallion Financial Corp',
  'names_previous': None},
 {'company_id': '1000228',
  'name_latest': 'Henry Schein Inc',
  'names_previous': None},
 {'company_id': '1000229',
  'name_latest': 'Core Laboratories N V',
  'names_previous': None},
 {'company_id': '1000230',
  'name_latest': 'Optical Cable Corp',
  'names_previous': None},
 {'company_id': '1000232',
  'name_latest': 'Kentucky Bancshares Inc /KY/',
  'names_previous': None},
 {'company_id': '1000278',
  'name_latest': 'Pacifichealth Laboratories Inc',
  'names_previous': None},
 {'company_id': '1000298',
  'name_latest': 'Impac Mortgage Holdings Inc',
  'names_previous': None},
 {'company_id': '1000459',
  'name_latest': 'Green Energy Group Inc.',
  'names_previous': None},
 {'company_id': '1000623',
  